In [22]:
from collections import Counter
import numpy as np
import pandas as pd

# ==============================================
# Fungsi Gini Impurity
# ==============================================
def gini_impurity(y):                                # hitung impurity Gini untuk vektor label y
    """Hitung impurity Gini dari label y"""
    hist = np.bincount(y)                            # jumlah kemunculan tiap kelas
    ps = hist / len(y)                               # proporsi tiap kelas
    return 1 - np.sum(ps ** 2)                       # rumus Gini = 1 - sum(p_k^2)

# ==============================================
# Struktur Node dan Decision Tree
# ==============================================
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature                        # index fitur yang dipakai split
        self.threshold = threshold                    # nilai ambang split
        self.left = left                              # anak kiri
        self.right = right                            # anak kanan
        self.value = value                            # nilai kelas bila leaf

    def is_leaf_node(self):
        return self.value is not None                 # leaf jika punya nilai kelas

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5, n_feats=None):
        self.min_samples_split = min_samples_split    # minimal sampel utk boleh displit
        self.max_depth = max_depth                    # kedalaman maksimum pohon
        self.n_feats = n_feats                        # jumlah fitur acak per node (None=pakai semua)
        self.root = None                              # root node
        self.feature_importances_ = None              # simpan "pentingnya" fitur (versi sederhana)

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])  # set n_feats
        self.root = self._grow_tree(X, y)             # bangun pohon mulai dari root
        self.feature_importances_ = self._compute_feature_importance(X)  # hitung importance (frekuensi split)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])  # telusuri pohon utk tiap sampel

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))                  # jumlah kelas unik pada node ini

        # Kriteria berhenti membelah node
        if (depth >= self.max_depth) or (n_labels == 1) or (n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)   # jadikan leaf dengan kelas mayoritas
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)  # pilih subset fitur (acak)

        # Cari split terbaik pada fitur-fitur terpilih
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        if best_feat is None:                         # jika tak ada gain, jadikan leaf
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)  # indeks kiri/kanan
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)   # rekursi ke kiri
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)# rekursi ke kanan
        return Node(best_feat, best_thresh, left, right)                    # kembalikan node internal

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1                                 # simpan information gain terbaik
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:                     # cek tiap fitur kandidat
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)           # coba semua nilai unik sebagai threshold
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)  # hitung gain
                if gain > best_gain:                   # simpan jika lebih baik
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh                 # kembalikan fitur & ambang terbaik

    def _information_gain(self, y, X_column, split_thresh):
        parent_loss = gini_impurity(y)                 # impurity node induk
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0                                   # jika salah satu sisi kosong → tak ada gain

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        g_l, g_r = gini_impurity(y[left_idxs]), gini_impurity(y[right_idxs])  # impurity anak
        child_loss = (n_l / n) * g_l + (n_r / n) * g_r                         # impurity tertimbang
        return parent_loss - child_loss                # IG = impurity parent − impurity anak

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()   # ke kiri jika ≤ threshold
        right_idxs = np.argwhere(X_column > split_thresh).flatten()   # ke kanan jika > threshold
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value                         # kembalikan kelas leaf
        if x[node.feature] <= node.threshold:         # bandingkan fitur sampel dengan ambang
            return self._traverse_tree(x, node.left)  # telusuri kiri
        return self._traverse_tree(x, node.right)     # atau kanan

    def _most_common_label(self, y):
        counter = Counter(y)                          # hitung frekuensi label
        most_common = counter.most_common(1)[0][0]    # ambil label terbanyak
        return most_common

    def _compute_feature_importance(self, X):
        """Hitung importance fitur berdasarkan frekuensi fitur digunakan"""
        importance = np.zeros(X.shape[1])             # inisialisasi vektor importance
        self._traverse_and_count(self.root, importance)  # tambah 1 tiap kali fitur jadi split
        if importance.sum() > 0:
            importance = importance / importance.sum()   # normalisasi ke proporsi
        return importance

    def _traverse_and_count(self, node, importance):
        if node is None or node.is_leaf_node():
            return                                   # berhenti di leaf
        importance[node.feature] += 1                # fitur ini dipakai split → tambah count
        self._traverse_and_count(node.left, importance)
        self._traverse_and_count(node.right, importance)

# ==============================================
# FUNGSI RFE (Recursive Feature Elimination)
# ==============================================
def recursive_feature_elimination(X, y, n_features_to_keep, feature_names):
    """Hapus fitur paling tidak penting sampai tersisa n_features_to_keep"""
    features = np.arange(X.shape[1])                  # indeks awal semua fitur
    feature_names = np.array(feature_names)

    while len(features) > n_features_to_keep:         # iterasi sampai sisa n_features_to_keep
        clf = DecisionTree(max_depth=5)               # latih pohon pada subset fitur saat ini
        clf.fit(X[:, features], y)
        importances = clf.feature_importances_        # importance versi 'frekuensi split'

        if importances.sum() == 0:                    # jika semua nol → hentikan
            print("⚠ Semua fitur sama pentingnya, proses dihentikan.")
            break

        least_important = np.argmin(importances)      # cari fitur paling tidak penting
        print(f" Menghapus fitur '{feature_names[features[least_important]]}' (indeks {features[least_important]}), sisa {len(features)-1} fitur.")
        features = np.delete(features, least_important)  # buang fitur tsb

    return features, feature_names[features]          # kembalikan indeks & nama fitur tersisa

# ==============================================
# MAIN PROGRAM
# ==============================================
if __name__ == "__main__":
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

    # Load dataset
    df = pd.read_csv("https://raw.githubusercontent.com/nurulaziz2004/utsai/main/datadt.csv")  # ambil data
    print("Data sample:")
    print(df.head())

    # Analisis & pembersihan data awal
    print("=== CEK NILAI KOSONG (NaN) SEBELUM DIBERSIHKAN ===")
    print(df.isna().sum())                            # cek NaN murni

    # Pastikan semua kolom numerik (antisipasi kolom object)
    for c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce') # coercion → non-numerik jadi NaN

    # Kolom fisiologis yang tak logis bernilai 0 → anggap missing
    cols_zero_means_missing = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

    print("\n=== JUMLAH NOL (yang dianggap missing) per kolom ===")
    print(df[cols_zero_means_missing].eq(0).sum())    # hitung banyaknya 0 per kolom

    # Ubah 0 → NaN agar dianggap missing
    df[cols_zero_means_missing] = df[cols_zero_means_missing].replace(0, np.nan)

    # (Opsional) juga bersihkan inf/-inf
    df = df.replace([np.inf, -np.inf], np.nan)

    # Strategi pembersihan:
    # A) Drop baris yang mengandung NaN pada kolom penting (mengurangi jumlah data)
    df = df.dropna(subset=cols_zero_means_missing)

    # B) Alternatif (komentari A, aktifkan ini) → imputasi median agar baris tetap banyak
    # for c in cols_zero_means_missing:
    #     df[c] = df[c].fillna(df[c].median())

    df = df.drop_duplicates()                         # hapus baris duplikat (jika ada)

    print("\n=== CEK NILAI KOSONG (NaN) SETELAH DIBERSIHKAN ===")
    print(df.isna().sum())
    print("\nJumlah data setelah pembersihan:", len(df))

    # Pisahkan fitur dan target (asumsi kolom terakhir = target)
    X = df.iloc[:, :-1].values                        # semua kolom kecuali terakhir → fitur
    y = df.iloc[:, -1].values.astype(int)             # kolom terakhir → label (int)
    feature_names = df.columns[:-1]                   # nama kolom fitur

    # Jalankan RFE untuk memilih 5 fitur terbaik
    selected_features, selected_names = recursive_feature_elimination(
        X, y, n_features_to_keep=5, feature_names=feature_names
    )
    print("\nFitur terpilih (berdasarkan nama):", list(selected_names))

    # Split data (pakai random_state agar replikasi konsisten; bisa tambah stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
        X[:, selected_features], y, test_size=0.3, random_state=42
        # , stratify=y                                # aktifkan ini agar proporsi kelas seimbang
    )

    # Buat & latih model Decision Tree
    clf = DecisionTree(max_depth=5)                   # pohon dengan kedalaman maks 5
    clf.fit(X_train, y_train)

    # Prediksi dan evaluasi
    y_pred = clf.predict(X_test)
    print("\n=== HASIL EVALUASI ===")
    print("Akurasi:", accuracy_score(y_test, y_pred)) # metrik akurasi
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))           # confusion matrix
    print("Classification Report:")
    print(classification_report(y_test, y_pred))      # precision/recall/f1 per kelas


Data sample:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
=== CEK NILAI KOSONG (NaN) SEBELUM DIBERSIHKAN ===
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0